In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path = 'all/ForestTypes/'
file_nm = 'training.csv'
file = pd.read_csv(path+file_nm)
file.head()

,class,b1,b2,b3,b4,b5,b6,b7,b8,b9,...,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9
0,d,39,36,57,91,59,101,93,27,60,...,-2.36,-18.41,-1.88,-6.43,-21.03,-1.60,-6.18,-22.50,-5.20,-7.86
1,h,84,30,57,112,51,98,92,26,62,...,-2.26,-16.27,-1.95,-6.25,-18.79,-1.99,-6.18,-23.41,-8.87,-10.83
2,s,53,25,49,99,51,93,84,26,58,...,-1.46,-15.92,-1.79,-4.64,-17.73,-0.48,-4.69,-19.97,-4.10,-7.07
3,s,59,26,49,103,47,92,82,25,56,...,2.68,-13.77,-2.53,-6.34,-22.03,-2.34,-6.60,-27.10,-7.99,-10.81
4,d,57,49,66,103,64,106,114,28,59,...,-2.94,-21.74,-1.64,-4.62,-23.74,-0.85,-5.50,-22.83,-2.74,-5.84


In [2]:
data = file.values
print (data)

[['d ' 39 36 ... -22.5 -5.2 -7.86]
 ['h ' 84 30 ... -23.41 -8.87 -10.83]
 ['s ' 53 25 ... -19.97 -4.1 -7.07]
 ...
 ['h ' 79 30 ... -23.32 -2.09 -4.13]
 ['h ' 69 27 ... -10.04 -0.74 -2.88]
 ['h ' 80 29 ... -20.91 -0.9 -3.7]]


In [3]:
class tree_node_class():
    def __init__(self, content, true_branch=None, false_branch=None):
        self.content  = content
        self.true_branch = true_branch
        self.false_branch = false_branch
    
    def is_leaf(self):
        return not (self.true_branch or self.false_branch)
    
class dtree_classification(object):
    node = tree_node_class
    
    def __init__(self):
        self.root = None
        self.size = 0
        
    def __len__(self):
        return self.size

In [4]:
def prepare_array_for_split(data, col_pos, label_pos):
    """
    """
    with_attr = np.array([row for row in data if not np.isnan(row[col_pos])])
    missing_attr = np.array([row for row in data if np.isnan(row[col_pos])])
    with_attr = with_attr[np.lexsort(np.fliplr(with_attr).T)]
    
    return with_attr, missing_attr

def numeric_get_split(data, col, split_value):
    """
    """
    left = np.array([rw for rw in data if rw[col] <= split_value])
    right = np.array([rw for rw in data if rw[col] > split_value])
    return left, right

# def discrete_get_split(data, attribute_set):
#     """
#     """
#     for split_val in attribute_set:
#         splits.append([rw[1] for rw in data if rw[0] == split_val])
#     return splits

In [5]:
def calc_entropy(groups, classes):
    """
    """
    entropy = 0
    total_inst = sum([len(each_gr[0]) for each_gr in groups])

    for each_gr in groups:
        score = 0
        
        if len(each_gr) == 0:
            continue
            
        label_col = each_gr[0]
        weight_col = each_gr[1].astype(np.float)
            
        for each_cl in classes:
            class_weight = [weight_col[i] 
                            for i in range(0, len(weight_col)) if label_col[i]==each_cl]
            p = sum(class_weight) / sum(weight_col)
            if p == 0:
                continue
            score += -p * np.log2(p)
            
        entropy += sum(weight_col) / total_inst * score
    return entropy

def calc_gini(groups, classes):
    """
    """
    gini = 0
    total_inst = sum([len(each_gr[0]) for each_gr in groups])
        
    for each_gr in groups:
        score = 0
        
        if len(each_gr[0]) == 0:
            continue
            
        label_col = each_gr[0]
        weight_col = each_gr[1].astype(np.float)

        for each_cl in classes:
            class_weight = [weight_col[i] 
                            for i in range(0, len(weight_col)) if label_col[i]==each_cl]
            p = sum(class_weight) / sum(weight_col)
            score += p * p
            
        gini += sum(weight_col) / total_inst * (1 - score)
    return gini

In [78]:
# dataset = [[2.771244718, 1.784783929, 0], 
#            [1.728571309, 1.169761413, 0], 
#            [3.678319846, 2.81281357, 0], 
#            [3.961043357, 2.61995032, 0], 
#            [2.999208922, 2.209014212, 0], 
#            [7.497545867, 3.162953546, 1], 
#            [9.00220326, 3.339047188, 1], 
#            [7.444542326, 0.476683375, 1], 
#            [10.12493903, 3.234550982, 1], 
#            [6.642287351, 3.319983761, 1]]

def add_weight(groups, label_pos):
    """
    """
    splits = []

    for each_gr in groups:
        label = [row[label_pos] for row in each_gr]
        with_weight = np.vstack((label, np.ones(len(each_gr))))
        splits.append(with_weight)

    return splits

def redistribute_missing(with_attr_grps, without_attr_grp):
    """
    """
    splits = []
    total_no_miss = sum([len(each) for each in with_attr_grps])

    for each_gr in with_attr_grps:
        if len(without_attr_grp) != 0:
            prob_weight = (len(each_gr) / total_no_miss) * np.ones(len(without_attr_grp))
            frac_attr = np.vstack((without_attr_grp, prob_weight))
            each_gr.append(frac_attr)
            
        splits.append(each_gr)

    return splits

def find_best_split(dataset, label_pos, homogeneity_measure='gini'):
    """
    """
    col_num = dataset.shape[1]
    
    if homogeneity_measure == 'entropy':
        weighted_label = add_weight([dataset], label_pos)
        total_entropy = calc_entropy([weighted_label], class_list)
    
    split = ()
    best_hg = 1
    for c in range(0, col_num):
        if c == label_pos:
            continue
        with_attr, _ = prepare_array_for_split(dataset, c, label_pos)

        if dataset_meta[c] == 'num':
            for r in range(1, len(with_attr)):
                split_val = (with_attr[r-1, c] + with_attr[r, c]) / 2
                left, right = numeric_get_split(with_attr, c, split_val)
                weighted_label_split = add_weight([left, right], label_pos)

                if homogeneity_measure == 'entropy':
                    info_gain = total_entropy - calc_entropy(weighted_label_split, class_list)
                    new_hg = info_gain

                    if new_hg > best_hg:
                        best_hg = new_hg
                        split = (c, split_val)
                else:
                    new_hg = calc_gini(weighted_label_split, class_list)

                    if new_hg < best_hg:
                        best_hg = new_hg
                        split = (c, split_val)
                        
#         else:
#             col_attr_set = list(set([row[0] for row in split_col]))
#             splits = discrete_get_split(split_col, col_attr_set)
            
#             if homogeneity_measure == 'entropy':
#                 info_gain = total_entropy - calc_entropy(splits, class_list)
#                 intrinsic_info = 
#             else:

    return split, best_hg

def construct_tree(root, data, min_leaf_size, homogeneity_measure='gini'):
    """
    """
    split_loc, score = find_best_split(data, label_pos, homogeneity_measure)
    left, right = numeric_get_split(data, split_loc[0], split_loc[1])

    if homogeneity_measure == 'entropy':
        pass
    else:
        left_hg = calc_gini(add_weight([left], label_pos), class_list)
        right_hg = calc_gini(add_weight([right], label_pos), class_list)
        min_hg = 0
    
    expression = 'Column ' + str(split_loc[0]) + ' (excl. label) <= ' + str(split_loc[1])
    print (expression)

    if len(root.content) == 0:
        root = tree_node_class(content=split_loc)
        
    if (len(left) <= min_leaf_size) or (left_hg == min_hg):
        root.true_branch = tree_node_class(content=left)
    else:
        root.true_branch = tree_node_class([])
        root.true_branch = construct_tree(root.true_branch, left, min_leaf_size)

    if (len(right) <= min_leaf_size) or (right_hg == min_hg):
        root.false_branch = tree_node_class(content=right)
    else:
        root.false_branch = tree_node_class([])
        root.false_branch = construct_tree(root.false_branch, right, min_leaf_size)
        
    return root

def print_tree(tree_root):
    """
    """
    global leaf_size_tot
    if tree_root is None:
        return
    print_tree(tree_root.true_branch)    
    print (tree_root.content)
    if tree_root.is_leaf():
        leaf_size_tot += len(tree_root.content)
    print_tree(tree_root.false_branch)

dataset = np.array(data)
print (dataset.shape)

label_pos = 0
min_leaf_size = 1

label_val = [row[label_pos] for row in dataset]
class_list = list(set(label_val))

dataset_meta = ['disc'] + ['num' for i in range(dataset.shape[1])]

r = tree_node_class([])
tree = construct_tree(r, dataset, min_leaf_size)

leaf_size_tot = 0
print ('Tree')
print_tree(tree)
print (leaf_size_tot)

(198, 28)
Column 2 (excl. label) <= 36.5
Column 10 (excl. label) <= 41.735
Column 22 (excl. label) <= -7.04
Column 1 (excl. label) <= 79.0
Column 1 (excl. label) <= 42.5
Column 1 (excl. label) <= 39.5
Column 13 (excl. label) <= -12.415
Column 5 (excl. label) <= 49.5
Column 8 (excl. label) <= 28.0
Column 1 (excl. label) <= 51.5
Column 7 (excl. label) <= 74.0
Tree
[['h ' 84 30 ... -23.41 -8.87 -10.83]
 ['h ' 85 28 ... -29.72 -1.94 -4.94]
 ['h ' 74 29 ... -17.2 -1.27 -5.42]
 ...
 ['h ' 79 30 ... -23.32 -2.09 -4.13]
 ['h ' 69 27 ... -10.04 -0.74 -2.88]
 ['h ' 80 29 ... -20.91 -0.9 -3.7]]
(22, -7.04)
[['h ' 67 31 55 102 47 89 81 23 56 36.04 16.24 34.0 -21.83 -23.89 -34.25
  -13.77 5.9 -2.28 -8.26 0.54 -1.29 -3.51 0.51 -0.59 -6.17 -0.81 -2.1]
 ['h ' 78 30 56 107 50 93 84 24 57 28.5 19.5 36.5 -23.34 -24.99 -35.98
  -6.17 6.02 -1.04 -12.8 0.84 -2.32 -6.14 -0.41 -1.18 -18.9 0.64 -3.3]]
(1, 79.0)
[['s ' 80 29 54 119 50 95 101 24 60 33.33 22.28 42.05 -19.36 -24.69
  -35.02 -25.24 6.32 -3.92 5.13 

In [100]:
file1_nm = 'testing.csv'
content = pd.read_csv(path+file1_nm)
content = content.values
print (content.shape)
print (content)

(325, 28)
[['d ' 67 51 ... -22.56 -5.53 -8.11]
 ['s ' 67 28 ... -22.2 -3.41 -6.57]
 ['s ' 63 26 ... -20.89 -3.96 -6.85]
 ...
 ['s ' 49 26 ... -24.5 -2.53 -4.97]
 ['s ' 55 26 ... -24.39 -2.21 -4.72]
 ['h ' 71 28 ... -12.74 -1.36 -3.63]]


In [111]:
def classify_from_tree(dtree, predict_cases, label_pos):
    """
    """
    output = []
    for case in predict_cases:
        case_tree = dtree
        while not case_tree.is_leaf():
            col_test, test = case_tree.content
            
            if case[col_test] <= test:
                case_tree = case_tree.true_branch
            else:
                case_tree = case_tree.false_branch
        
        label = [row[label_pos] for row in case_tree.content]
        most_common_value = max(map(lambda x: (x, label.count(x)), set(label)))[0]
        output.append(most_common_value)
    return output

classification = classify_from_tree(tree, content, label_pos)
print (classification)
true_label = [row[label_pos] for row in content]
diff = [predict == label for predict, label in zip(classification, true_label)]
acc = sum(diff) / content.shape[0] * 100
print ('Accuracy {}'.format(acc))

['o ', 's ', 's ', 'd ', 's ', 'o ', 'h ', 'd ', 'h ', 'd ', 's ', 'o ', 's ', 'h ', 'o ', 's ', 'o ', 'd ', 's ', 'd ', 'o ', 's ', 'd ', 's ', 's ', 's ', 's ', 's ', 's ', 'd ', 's ', 'o ', 's ', 's ', 'o ', 'd ', 'o ', 'd ', 's ', 'o ', 'd ', 'h ', 'o ', 'o ', 's ', 'o ', 's ', 's ', 'h ', 's ', 'o ', 'o ', 'h ', 'h ', 'd ', 's ', 'h ', 'o ', 's ', 's ', 's ', 'd ', 's ', 's ', 's ', 's ', 'd ', 'd ', 'd ', 's ', 's ', 'd ', 'o ', 's ', 'h ', 's ', 's ', 's ', 's ', 's ', 'd ', 's ', 's ', 'h ', 'd ', 's ', 's ', 'o ', 'd ', 's ', 'o ', 'd ', 's ', 's ', 's ', 'd ', 'd ', 's ', 'd ', 'd ', 's ', 'h ', 's ', 's ', 's ', 's ', 'o ', 'o ', 'o ', 's ', 'o ', 's ', 'o ', 's ', 's ', 'o ', 'o ', 's ', 's ', 'h ', 'd ', 'h ', 'd ', 's ', 's ', 's ', 'd ', 's ', 'd ', 'o ', 'd ', 'd ', 's ', 's ', 'd ', 's ', 'o ', 'd ', 's ', 's ', 'd ', 'd ', 's ', 'o ', 's ', 'd ', 's ', 'd ', 's ', 'h ', 'h ', 'd ', 'o ', 'd ', 'o ', 'd ', 'o ', 'd ', 'h ', 'd ', 's ', 'd ', 'd ', 's ', 's ', 's ', 'd 